# Canadian Energy Solutions

This is a simple notebook that allows us to explore some of the underlying problems in the Canadian energy space. The objective is to identify sectors that need assistance, and potentially identify companies that can fill those gaps.

If we were to simplify the Canadian energy problem, we can say that we need to reach a scenario where two simple constraints are met:

`supply > demand && net_emissions <= 0`

In reality, this is a much more complex equation since if we were to sum the entirety of Canada's production, and subtract demand, we would be glossing over regional differences and distances.

For the sake of this document, we will start by assuming two things:
- A province must be fully sufficient. That is to say that it must not depend on neighbouring provinces to provide electricity.
- Emissions across the country must be net. One province may pollute, as long as neighbouring provinces can assist with the reduction (it's all one atmosphere.)

In [47]:
%matplotlib inline

# import csv
import ssl

# We're disabling SSL Cert Verification for these data examples
ssl._create_default_https_context = ssl._create_unverified_context

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [48]:
# Reset all data
butane_data = None
coal_data = None
crude_oil_data = None
electricity_data = None
ethane_data = None
natural_gas_data = None
pentanes_data = None
propane_data = None
end_use_demand_data = None

In [49]:
# Let's import our data

# This uses Canada's Energy Futures 2020 Dataset. This dataset looks far into the future and gives us a good idea of whether
# we're heading in the right direction or not. This isn't necessarily a 100% accurate, as some data sources are limited but it's
# a good start.
#
# >  The Energy Futures series explores how possible energy futures might unfold for Canadians over the long term.
# >  Canada’s Energy Future 2020: Energy Supply and Demand Projections to 2050 (EF2020) is our latest long-term energy outlook.
# >  It is the first outlook in the series to provide projections to 2050.
# >  It covers all energy commodities, and all provinces and territories.
# >  We use economic and energy models to develop this outlook. We also make assumptions about technology,
# >  energy and climate policies, energy markets, human behaviour and the economy.
#

# https://open.canada.ca/data/en/dataset/bba41250-261a-4f3b-9ce8-db44d9a0f725

# Butane
print("Importing Butane Data...")
butane_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/butanes-2020.csv")

# Coal
print("Importing Coal Data...")
coal_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/coal-2020.csv")

# Crude Oil
print("Importing Crude Oil Data...")
crude_oil_production_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/crude-oil-production-2020.csv")

print("Importing Crude Oil Export Data...")
crude_oil_export_data = pd.read_csv("https://www.cer-rec.gc.ca/open/imports-exports/crude-oil-exports-by-type-annual.csv")

# Electricity Generation
print("Importing Electricity Generation Data...")
electricity_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/electricity-generation-2020.csv")

# Ethane
print("Importing Ethane Data...")
ethane_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/ethane-2020.csv")

# Natural Gas Production
print("Importing Natural Gas Data...")
natural_gas_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/natural-gas-production-2020.csv")

print("Importing Natural Gas Export Data...")
natural_gas_export_data = pd.read_csv("https://www.cer-rec.gc.ca/open/imports-exports/natural-gas-exports-and-imports-annual.csv")

# Pentanes
print("Importing Pentanes Data...")
pentanes_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/pentanes-2020.csv")

# Propane
print("Importing Propane Data...")
propane_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/propane-2020.csv")

# Overall Import/Export Data
print("Importing CER Imports/Exports Data...")
cer_imports_exports = pd.read_csv("https://apps2.cer-rec.gc.ca/imports-exports/data/CER_imports_exports_data.csv")

# Primary Demand
print("Importing Primary Demand...")
primary_demand_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/primary-energy-demand-2020.csv")

# End Use Demand
print("Importing End Use Demand Data...")
end_use_demand_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/end-use-demand-2020.csv")

# Electric power generation, monthly generation by type of electricity
# Source: https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=2510001501
#
# https://www150.statcan.gc.ca/t1/tbl1/#?pid=25100015&file=2510001501-eng.csv

print("Done!")

Importing Butane Data...
Importing Coal Data...
Importing Crude Oil Data...
Importing Crude Oil Export Data...
Importing Electricity Generation Data...
Importing Ethane Data...
Importing Natural Gas Data...
Importing Natural Gas Export Data...
Importing Pentanes Data...
Importing Propane Data...
Importing CER Imports/Exports Data...
Importing Primary Demand...
Importing End Use Demand Data...
Done!


In [50]:
# Data setup
layer_1_production_and_imports = {}
layer_2_primary_energy = {}
layer_3_secondary_energy = {}
layer_4_industry = {}
layer_5_industry_end_use = {}

In [51]:
# Helper Methods
# Conversion Data: https://apps.cer-rec.gc.ca/Conversion/conversion-tables.aspx?GoCTemplateCulture=en-CA

gj_to_pj = 1000000

def gigajoule_to_quad(gj):
  return -1

# Butane, in Cubic Metres
def butane_to_pj(butane):
  return butane * 28.62 / gj_to_pj

# Coal, in tons
# Coal has four different "types" each resulting in different energy densities.
#   Anthracite: 27.70 GJ
#   Bituminous: 27.60 GJ
#   Lignite: 14.40 GJ
#   Subbituminous: 18.80 GJ
def coal_to_pj(coal):
  return coal * 27.60 / gj_to_pj

# Light Crude Oil, in m³
#   Light	1.0 Cubic metres (m³)	38.51 Gigajoules (GJ)
def light_crude_oil_to_pj(crude_oil):
  return crude_oil * 38.41 / gj_to_pj

# Heavy Crude Oil, in m³
#   Heavy	1.0 Cubic metres (m³)	40.90 Gigajoules (GJ)
def heavy_crude_oil_to_pj(crude_oil):
  return crude_oil * 40.90 / gj_to_pj

# C5+, in m³
#   Pentanes plus	1.0 Cubic metres (m³)	35.17 Gigajoules (GJ)
def pentanes_plus_crude_oil_to_pj(crude_oil):
  return crude_oil * 35.17 / gj_to_pj

# Synthetic Crude Oil, in m³
#   Synthetic crude oil	1.0 Cubic metres (m³)	39.40 Gigajoules (GJ)
def synthetic_crude_oil_to_pj(crude_oil):
  return crude_oil * 39.40 / gj_to_pj

# Bitument Crude Oil
#   Bitumen	1.0 Cubic metres (m³)	42.80 Gigajoules (GJ)
def bitumen_crude_oil_to_pj(crude_oil):
  return crude_oil * 42.80 / gj_to_pj

# Ethane, in Cubic Metres
def ethane_to_pj(ethane):
  return ethane * 18.36 / gj_to_pj

# Natural Gas, in Cubic Metres
def natural_gas_to_pj(natural_gas):
  return natural_gas * 0.0373 / gj_to_pj

def pentanes_to_pj(pentanes):
  return pentanes * 35.17 / gj_to_pj

def propane_to_pj(propane):
  return propane * 25.53 / gj_to_pj

# Using https://www.cesarnet.ca/visualization/sankey-diagrams-canadas-energy-systems 's conversion,
# > The primary heat energy generated by uranium depends on the reactor technology being used, and is much higher in reactors that use enriched uranium than in the CANDU natural uranium reactors used in Canada. The typical heat rate for uranium in a CANDU is 7700 MW-days(thermal)/tonne U, which converts to 0.665 PJ/tonne U, and this is the the conversion factor used in this Sankey diagram for estimating the primary energy content of uranium.
def uranium_to_pj(uranium):
  return uranium * 0.665

def gwh_to_pj(gwh):
  return gwh * 0.0036

In [52]:
def extract_butane_data(layer_1, data, year):
  butane_data = data[(data["Year"] == year) & (data["Unit"] == "Thousand Cubic Metres per day")]

  # Butane data is "per day". We assume 365 day averages.
  layer_1["butane_imports"] = butane_to_pj(1000 * 365 * butane_data[(butane_data["Variable"] == "Imports to AB from U.S.")]["Value"].sum())
  layer_1["butane_production"] = butane_to_pj(1000 * 365 * butane_data[(butane_data["Variable"].isin(["Production from Gas Processing", "Production from Oil Sands off-gas"]))]["Value"].sum())
  layer_1["butane_exports"] = butane_to_pj(1000 * 365 * butane_data[(butane_data["Variable"] == "Projected Exports")]["Value"].sum())

In [53]:
def extract_coal_data(layer_1, data, year):
  coal_data = data[data["Year"] == year]

  # These values are all in kilotons
  layer_1["coal_imports"] = coal_to_pj(1000 * coal_data[(coal_data["Variable"] == "Total") & (coal_data["Type"] == "Imports")].iloc[0]["Value"])
  layer_1["coal_production"] = coal_to_pj(1000 * coal_data[(coal_data["Variable"] == "Total") & (coal_data["Type"] == "Production")].iloc[0]["Value"])
  layer_1["coal_exports"] = coal_to_pj(1000 * coal_data[(coal_data["Variable"] == "Total") & (coal_data["Type"] == "Exports")].iloc[0]["Value"])

  #TODO: Use Domestic Demand for other layers

In [54]:
def extract_crude_oil_data(layer_1, data, export_data, import_export_data, year):
  crude_oil_data = data[(data["Scenario"] == "Reference") & (data["Year"] == year) & (data["Region"] == "Canada") & (data["Unit"] == "Thousand Cubic Metres per day")]
  crude_oil_export_data = export_data[(export_data["Year"] == year) & (export_data["Oil Type"] == "Total")]

  #TODO: THis data set is lacking 2020Q4 so we can't do an accurate picture o 2020 imports.
  crude_oil_import_data = import_export_data[(import_export_data["Period"].isin([f'{year}Q1', f'{year}Q2', f'{year}Q3', f'{year}Q4'])) & (import_export_data["Product"] == "Crude Oil") & (import_export_data["Activity"] == "Imports") & (import_export_data["Value"] != "Confidential")]

  # We're missing data from 2020Q4, so take the mean of Q1, Q2, and Q3
  crude_oil_import_data = import_export_data[(import_export_data["Product"] == "Crude Oil") & (import_export_data["Activity"] == "Imports") & (import_export_data["Value"] != "Confidential")]

  import_q1 = crude_oil_import_data[crude_oil_import_data["Period"] == f'{year}Q1']["Value"].astype(float).sum()
  import_q2 = crude_oil_import_data[crude_oil_import_data["Period"] == f'{year}Q2']["Value"].astype(float).sum()
  import_q3 = crude_oil_import_data[crude_oil_import_data["Period"] == f'{year}Q3']["Value"].astype(float).sum()

  # TODO: Get real data. Average is not a great representation.
  # TODO: We aren't told what kind of crude oil is imported (marked as N/A) therefore this is a guess and we just label it as heavy as a "best effort".
  average_import = (import_q1 + import_q2 + import_q3) / 3
  layer_1["crude_oil_imports"] = heavy_crude_oil_to_pj(1000 * 365 * average_import)

  heavy_crude_oil = crude_oil_data[crude_oil_data["Variable"] == "Conventional Heavy"].iloc[0]["value"]
  light_crude_oil = crude_oil_data[crude_oil_data["Variable"] == "Conventional Light"].iloc[0]["value"]
  pentanes_plus_crude_oil = crude_oil_data[crude_oil_data["Variable"] == "C5+"].iloc[0]["value"]
  synthetic_crude_oil = crude_oil_data[crude_oil_data["Variable"] == "(Upgraded Bitumen)"].iloc[0]["value"]
  bitumen_crude_oil = crude_oil_data[crude_oil_data["Variable"] == "Mined Bitumen"].iloc[0]["value"] + crude_oil_data[crude_oil_data["Variable"] == "In Situ Bitumen"].iloc[0]["value"] - synthetic_crude_oil
  # TODO: We're currently ignoring Field Condensate (mostly made up of NGL) -- We need to determine if this number is separate from the Butane/Pentane/etc. production

  layer_1["crude_oil_production"] = heavy_crude_oil_to_pj(1000 * 365 * heavy_crude_oil) + light_crude_oil_to_pj(1000 * 365 * light_crude_oil) + pentanes_plus_crude_oil_to_pj(1000 * 365 * pentanes_plus_crude_oil) + bitumen_crude_oil_to_pj(1000 * 365 * bitumen_crude_oil) + synthetic_crude_oil_to_pj(1000 * 365 * synthetic_crude_oil)

  layer_1["crude_oil_exports"] = heavy_crude_oil_to_pj(365 * crude_oil_export_data["Volume (m3/d)"].iloc[0])

In [55]:
def extract_electricity_data(layer_2, data, year):
  electricity_data = data[(data["Scenario"] == "Reference") & (data["Year"] == year) & (data["Region"] == "Canada")]

  # Values in GW/h

  #TODO: Convert units
  layer_2["hydro_wave_tidal"] = gwh_to_pj(electricity_data[(electricity_data["Variable"] == "Hydro / Wave / Tidal")].iloc[0]["Value"])
  layer_2["natural_gas"] = gwh_to_pj(electricity_data[(electricity_data["Variable"] == "Natural Gas")].iloc[0]["Value"])
  layer_2["oil"] = gwh_to_pj(electricity_data[(electricity_data["Variable"] == "Oil")].iloc[0]["Value"])
  layer_2["biomass_geothermal"] = gwh_to_pj(electricity_data[(electricity_data["Variable"] == "Biomass / Geothermal")].iloc[0]["Value"])

  #TODO: Can this not be split?
  layer_2["coal_and_coke"] = gwh_to_pj(electricity_data[(electricity_data["Variable"] == "Coal & Coke")].iloc[0]["Value"])
  layer_2["uranium"] = gwh_to_pj(electricity_data[(electricity_data["Variable"] == "Uranium")].iloc[0]["Value"])
  layer_2["solar"] = gwh_to_pj(electricity_data[(electricity_data["Variable"] == "Solar")].iloc[0]["Value"])
  layer_2["wind"] = gwh_to_pj(electricity_data[(electricity_data["Variable"] == "Wind")].iloc[0]["Value"])

In [56]:
def extract_ethane_data(layer_1, data, year):
  ethane_data = data[(data["Year"] == year) & (data["Unit"] == "Thousand Cubic Metres per day")]

  layer_1["ethane_imports"] = ethane_to_pj(1000 * 365 * ethane_data[ethane_data["Variable"].isin(["Imports to AB from U.S.", "Imports to ON from U.S."])]["Value"].sum())
  layer_1["ethane_production"] = ethane_to_pj(1000 * 365 * ethane_data[ethane_data["Variable"].isin(["Production from Gas Processing", "Production from Oil Sands off-gas"])]["Value"].sum())
  layer_1["ethane_exports"] = ethane_to_pj(1000 * 365 * ethane_data[ethane_data["Variable"] == "Exports"].iloc[0]["Value"])

In [57]:
def extract_natural_gas_data(layer_1, data, export_data, year):
  natural_gas_data = data[(data["Year"] == year) & (data["Region"] == "Canada") & (data["Unit"] == "Million Cubic Metres per day")]
  natural_gas_export_data = export_data[(export_data["Year"] == year)]

  #TODO: Confirm http://www.cer-rec.gc.ca/en/data-analysis/energy-commodities/natural-gas/report/natural-gas-summary/natural-gas-annual-trade-summary.html
  layer_1["natural_gas_imports"] = natural_gas_to_pj(1000 * natural_gas_export_data[(natural_gas_export_data["Activity"] == "Imports")]["Volume (10^3 m3)"].sum())
  layer_1["natural_gas_production"] = natural_gas_to_pj(1000000 * 365 * natural_gas_data.iloc[0]["Value"])
  layer_1["natural_gas_exports"] = natural_gas_to_pj(1000 * natural_gas_export_data[(natural_gas_export_data["Activity"] == "Exports")]["Volume (10^3 m3)"].sum())

In [58]:
def extract_pentanes_data(layer_1, data, year):  
  pentanes_data = data[(data["Year"] == year) & (data["Unit"] == "Thousand Cubic Metres per day")]

  layer_1["pentane_imports"] = pentanes_to_pj(1000 * 365 * pentanes_data[pentanes_data["Variable"] == "Net Imports"].iloc[0]["Value"])
  layer_1["pentane_production"] = pentanes_to_pj(1000 * 365 * pentanes_data[pentanes_data["Variable"].isin(["Produciton from Refineries", "Production from Gas Processing - Pentanes Plus", "Production from Gas Wells - Liquid Condensate"])]["Value"].sum())
  layer_1["pentane_exports"] = pentanes_to_pj(1000 * 365 * pentanes_data[pentanes_data["Variable"] == "Net Exports"].iloc[0]["Value"])

In [59]:
def extract_propane_data(layer_1, data, year):
  propane_data = data[(data["Year"] == year) & (data["Unit"] == "Thousand Cubic Metres per day")]

  layer_1["propane_imports"] = propane_to_pj(1000 * 365 * propane_data[propane_data["Variable"] == "Imports"].iloc[0]["Value"])
  layer_1["propane_production"] = propane_to_pj(1000 * 365 * propane_data[propane_data["Variable"].isin(["Production from Gas Processing", "Production from Oil Sands off-gas", "Production from Refineries"])]["Value"].sum())
  layer_1["propane_exports"] = propane_to_pj(1000 * 365 * propane_data[propane_data["Variable"] == "Projected Exports"].iloc[0]["Value"])

In [60]:
def extract_uranium_data(layer_1):
  # Uranium
  # -------
  # Sources
  # 1. https://www.world-nuclear.org/information-library/country-profiles/countries-a-f/canada-uranium.aspx
  # 2. https://www.cameco.com/invest/financial-information/annual-reports/2020
  # 3. https://www.cameco.com/businesses/uranium-operations/canada/cigar-lake
  # 4. https://www.nrcan.gc.ca/energy/energy-sources-distribution/uranium-nuclear-energy/uranium-canada/
  # 5. https://www.nrcan.gc.ca/energy/energy-sources-distribution/uranium-nuclear-energy/uranium-canada/about-uranium/7695
  # 6. https://www.cameco.com/invest/overview
  # 7. https://www.world-nuclear.org/information-library/nuclear-fuel-cycle/mining-of-uranium/world-uranium-mining-production.aspx
  # 
  # We have limited data here, but the totals per year are: (tonnes U3O8) [1]
  # 2019: 6,938 [7]
  # 2018: 7,001
  # 2017: 13,116
  # 2016: 14,039
  # 2015: 13,325
  # 2014: 9,134
  # 2013: 9,331

  # However if we look at Cameco (owner of Canada's Cigar Lake Uranium facility):
  # - Cameco represents 50% ownership of the Cigar Lake facility and their 2020 share was 5,000,000 pounds -- thus we can estimate that the Cigar Lake production was 10m lbs. [3]
  # - 10m lbs = 5000 tons
  # - Cigar Lake represented 100% of Canada's Uranium production in 2019. [1]
  uranium_production_2020 = 5000

  layer_1["uranium_imports"] = 0
  layer_1["uranium_production"] = uranium_to_pj(uranium_production_2020) #tons
  layer_1["uranium_exports"] = uranium_to_pj(0.85 * uranium_production_2020)
  layer_1["uranium_used"] = uranium_to_pj(0.15 * uranium_production_2020)
  # We don't need to cover this number, since it may actually represent previous-year stores being used as well
  # [6] https://www.cameco.com/invest/overview (Blind River)
  # layer_1["uranium_processing"] = 11.7m kgU

In [61]:
def extract_layer_1(layer, year):
  extract_butane_data(layer, butane_data, year)
  extract_coal_data(layer, coal_data, year)
  extract_crude_oil_data(layer, crude_oil_production_data, crude_oil_export_data, cer_imports_exports, year)
  extract_ethane_data(layer, ethane_data, year)
  extract_natural_gas_data(layer, natural_gas_data, natural_gas_export_data, year)
  extract_pentanes_data(layer, pentanes_data, year)
  extract_propane_data(layer, propane_data, year)
  extract_uranium_data(layer)

def extract_layer_2(layer, year):
  extract_electricity_data(layer, electricity_data, year)

extract_layer_1(layer_1_production_and_imports, 2020)
extract_layer_2(layer_2_primary_energy, 2020)
display(layer_1_production_and_imports)

display(layer_2_primary_energy)

{'butane_imports': 22.081154525028,
 'butane_production': 512.9647136365068,
 'butane_exports': 160.0354700761122,
 'coal_imports': 162.66573448320003,
 'coal_production': 1245.751417116,
 'coal_exports': 896.6025652440001,
 'crude_oil_imports': 1294.948605391703,
 'crude_oil_production': 10279.083898502427,
 'crude_oil_exports': 8713.894234706087,
 'ethane_imports': 230.099958365724,
 'ethane_production': 477.62991630643666,
 'ethane_exports': 0.0,
 'natural_gas_imports': 1693.4744250955812,
 'natural_gas_production': 6062.15880664835,
 'natural_gas_exports': 5290.62272400377,
 'pentane_imports': 395.79014831042747,
 'pentane_production': 1857.7547574999141,
 'pentane_exports': 0.0,
 'propane_imports': 21.6585977511597,
 'propane_production': 872.9869648030601,
 'propane_exports': 298.146518487105,
 'uranium_imports': 0,
 'uranium_production': 3325.0,
 'uranium_exports': 2826.25,
 'uranium_used': 498.75}

{'hydro_wave_tidal': 1385.72064,
 'natural_gas': 252.60242399999998,
 'oil': 9.28350036,
 'biomass_geothermal': 29.512054799999998,
 'coal_and_coke': 121.35540095999998,
 'uranium': 320.65128,
 'solar': 11.438586,
 'wind': 124.822944}

In [62]:
colours = {
  "Butane Production" : "#f94144",
  "Butane Imports" : "#f94144",
  "Butane Exports" : "#f94144",

  "Coal Production" : "#f3722c",
  "Coal Imports" : "#f3722c",
  "Primary Coal" : "#f3722c",

  "Crude Oil Production" : "#f8961e",
  "Crude Oil Imports" : "#f8961e",
  "Primary RPP" : "#f8961e",
  
  "Ethane Production" : "#f9844a",
  "Ethane Imports" : "#f9844a",
  "Ethane Exports" : "#f9844a",
  
  "Natural Gas Production" : "#f9c74f",
  "Natural Gas Imports" : "#f9c74f",
  
  "Primary Natural Gas" : "#f9c74f",
  
  "Pentane Production" : "#90be6d",
  "Pentane Imports" : "#90be6d",
  "Pentane Exports" : "#90be6d",
  
  "Propane Production" : "#4d908e",
  "Propane Imports" : "#4d908e",
  "Propane Exports" : "#4d908e",

  "Uranium Production" : "#577590",
  "Uranium Imports" : "#577590",
  "Primary Uranium" : "#577590",

  "Hydroelectric Energy Production" : "#277da1",
  "Primary Hydro" : "#277da1",

  "Wind Energy Production" : "#f3722c",
  "Primary Wind" : "#f3722c",

  "TODO ???" : "black",
  "TODO ????" : "black",

  "Primary NGL" : "#f3722c",
  "Primary Other Renewables and Landfill Gas" : "black",
}

In [71]:
from sankey import Sankey
sankey = Sankey()

# Natural Gas Liquids
# Butane
sankey.add_edge("Butane Production", "Primary NGL", layer_1_production_and_imports["butane_production"])
sankey.add_edge("Butane Imports", "Primary NGL", layer_1_production_and_imports["butane_imports"])

sankey.add_edge("Primary NGL", "Butane Exports", layer_1_production_and_imports["butane_exports"])
sankey.add_edge("Butane Exports", "Exports", layer_1_production_and_imports["butane_exports"])

# Ethane
sankey.add_edge("Ethane Production", "Primary NGL", layer_1_production_and_imports["ethane_production"])
sankey.add_edge("Ethane Imports", "Primary NGL", layer_1_production_and_imports["ethane_imports"])

sankey.add_edge("Primary NGL", "Ethane Exports", layer_1_production_and_imports["ethane_exports"])
sankey.add_edge("Ethane Exports", "Exports", layer_1_production_and_imports["ethane_exports"])

# Pentane
sankey.add_edge("Pentane Production", "Primary NGL", layer_1_production_and_imports["pentane_production"])
sankey.add_edge("Pentane Imports", "Primary NGL", layer_1_production_and_imports["pentane_imports"])

sankey.add_edge("Primary NGL", "Pentane Exports", layer_1_production_and_imports["pentane_exports"])
sankey.add_edge("Pentane Exports", "Exports", layer_1_production_and_imports["pentane_exports"])

# Propane
sankey.add_edge("Propane Production", "Primary NGL", layer_1_production_and_imports["propane_production"])
sankey.add_edge("Propane Imports", "Primary NGL", layer_1_production_and_imports["propane_imports"])

sankey.add_edge("Primary NGL", "Propane Exports", layer_1_production_and_imports["propane_exports"])
sankey.add_edge("Propane Exports", "Exports", layer_1_production_and_imports["propane_exports"])

sankey.add_edge("Primary NGL", "TODO Produced & Distributed Energy", 1.0)

# ---

# Natural Gas
sankey.add_edge("Natural Gas Production", "Primary Natural Gas", layer_1_production_and_imports["natural_gas_production"])
sankey.add_edge("Natural Gas Imports", "Primary Natural Gas", layer_1_production_and_imports["natural_gas_imports"])

sankey.add_edge("Primary Natural Gas", "Exports", layer_1_production_and_imports["natural_gas_exports"])
sankey.add_edge("Primary Natural Gas", "TODO Produced & Distributed Energy", 1.0)

# ---

# Coal
sankey.add_edge("Coal Production", "Primary Coal", layer_1_production_and_imports["coal_production"])
sankey.add_edge("Coal Imports", "Primary Coal", layer_1_production_and_imports["coal_imports"])

sankey.add_edge("Primary Coal", "Exports", layer_1_production_and_imports["coal_exports"])
sankey.add_edge("Primary Coal", "TODO Produced & Distributed Energy", 1.0)

# Crude Oil
sankey.add_edge("Crude Oil Production", "Primary RPP", layer_1_production_and_imports["crude_oil_production"])
sankey.add_edge("Crude Oil Imports", "Primary RPP", layer_1_production_and_imports["crude_oil_imports"])

sankey.add_edge("Primary RPP", "Exports", layer_1_production_and_imports["crude_oil_exports"])
sankey.add_edge("Primary RPP", "TODO Produced & Distributed Energy", 1.0)

# Uranium
sankey.add_edge("Uranium Production", "Primary Uranium", layer_1_production_and_imports["uranium_production"])
sankey.add_edge("Uranium Imports", "Primary Uranium", layer_1_production_and_imports["uranium_imports"])

sankey.add_edge("Primary Uranium", "Exports", layer_1_production_and_imports["uranium_exports"])
sankey.add_edge("Primary Uranium", "Reserves or Error", layer_1_production_and_imports["uranium_production"] - layer_1_production_and_imports["uranium_exports"] - layer_2_primary_energy["uranium"])
# Renewables
sankey.add_edge("Wind Energy Production", "Primary Wind", layer_2_primary_energy["wind"])
sankey.add_edge("Hydroelectric Energy Production", "Primary Hydro", layer_2_primary_energy["hydro_wave_tidal"])
sankey.add_edge("Solar Energy Production", "Primary Solar", layer_2_primary_energy["solar"])

#TODO: This needs to be properly calculated
sankey.add_edge("Biomass & Geothermal", "Primary Other Renewables and Landfill Gas", layer_2_primary_energy["biomass_geothermal"])

# Other
sankey.add_edge("TODO ???", "TODO ????", 1.0)
sankey.add_edge("TODO ????", "Primary Other Renewables and Landfill Gas", 1.0)

# Electricity
sankey.add_edge("Primary Coal", "Electricity", layer_2_primary_energy["coal_and_coke"])
sankey.add_edge("Primary Hydro", "Electricity", layer_2_primary_energy["hydro_wave_tidal"])

sankey.add_edge("Primary Natural Gas", "Electricity", layer_2_primary_energy["natural_gas"])
sankey.add_edge("Primary Uranium", "Electricity", layer_2_primary_energy["uranium"])
sankey.add_edge("Primary Other Renewables and Landfill Gas", "Electricity", layer_2_primary_energy["biomass_geothermal"])
sankey.add_edge("Primary Solar", "Electricity", layer_2_primary_energy["solar"])
sankey.add_edge("Primary RPP", "Electricity", layer_2_primary_energy["oil"])
sankey.add_edge("Primary Wind", "Electricity", layer_2_primary_energy["wind"])

sankey.set_colours(colours)
sankey.validate()

sankey.render("2020 Canadian Energy Future Sankey Diagram")


AttributeError: 'Sankey' object has no attribute 'validate'

In [64]:
end_use_demand_data_2020 = end_use_demand_data[(end_use_demand_data["Year"] == 2020) & (end_use_demand_data["Region"] == "Canada")]
display(end_use_demand_data_2020)

,Scenario,Unit,Region,Variable,Type,Year,Value,Sector
642,Reference,NaN,Canada,Total End-Use,NaN,2020,1760.9776,Residential
643,Reference,NaN,Canada,Electricity,NaN,2020,688.3434,Residential
644,Reference,NaN,Canada,Natural Gas,NaN,2020,816.7201,Residential
645,Reference,NaN,Canada,RPP,NaN,2020,76.2924,Residential
646,Reference,NaN,Canada,Biofuels & Emerging Energy,NaN,2020,179.6216,Residential
...,...,...,...,...,...,...,...,...
40571,Evolving,NaN,Canada,Natural Gas,NaN,2020,4294.0171,Total End-Use
40572,Evolving,NaN,Canada,RPP,NaN,2020,4444.1012,Total End-Use
40573,Evolving,NaN,Canada,Biofuels & Emerging Energy,NaN,2020,627.4849,Total End-Use
40574,Evolving,NaN,Canada,Other,NaN,2020,120.6927,Total End-Use


# Production 2019

This section outlines how our energy was produced.

Canada produces *energy* in the following forms:

## Primary Energy
1. Coal
2. Crude Oil
3. Natural Gas
4. Natural Gas liquids (NGL)
5. Hydro
6. Nuclear
7. Steam
8. Wind
9. Solar

## Secondary Energy
8. Coke
9. Coke Oven Gas
10. Refined Petroleum Products
11. Secondary Electricity Sources (i.e. Thermal)

Source: https://www150.statcan.gc.ca/t1/tbl1/en/cv.action?pid=2510003001

See: data/Canada/2510003001-eng.csv

**Note**: All energy values are to be shown as [quads](https://en.wikipedia.org/wiki/Quad_(unit)), where one quad is equal to 10^16 BTU or 1.055 * 10^18 joules (1.055 exajoules)

> Some common types of an energy carrier approximately equal to 1 quad are:

    8,007,000,000 gallons (US) of gasoline
    293,071,000,000 kilowatt-hours (kWh)
    293.07 terawatt-hours (TWh)
    33.434 gigawatt-years (GWy)
    36,000,000 tonnes of coal
    970,434,000,000 cubic feet of natural gas
    5,996,000,000 UK gallons of diesel oil
    25,200,000 tonnes of oil
    252,000,000 tonnes of TNT or five times the energy of the Tsar Bomba nuclear test
    13.3 tonnes of uranium-235


In [65]:
## Global

production = {}
imports = {}
exports = {}

In [66]:
## Coal

kiloton_to_tons = 1000
quad_to_kiloton_of_coal = 36000000

def kiloton_of_coal_to_quad(kilotons):
  return kilotons * kiloton_to_tons / quad_to_kiloton_of_coal

production["coal"] = kiloton_of_coal_to_quad(50577.50)
imports["coal"] = kiloton_of_coal_to_quad(0)
exports["coal"] = kiloton_of_coal_to_quad(0)

In [67]:
## Crude Oil

megalitre_of_crude_oil = 1000000
barrel_of_oil_in_litres = 158.9873
quad_to_boe = 180135869.19434

def megalitre_to_quad(megalitres):
  return megalitres * megalitre_of_crude_oil / barrel_of_oil_in_litres / quad_to_boe

production["crude_oil"] = megalitre_to_quad(272186.3)
imports["crude_oil"] = megalitre_to_quad(45484.7)
exports["crude_oil"] = megalitre_to_quad(219364.7)

In [68]:
## Natural Gas

# Gigalitre is 1000 * 1000 * 1000 litres. 1000 litres in a cubic metre.
gigalitres_to_cubic_metres = 1000000
# 1 Gm3 NG = 0.035687347874265 quad
cubic_meter_to_quad = 0.035687347874265 / 1000000000

def gigalitre_to_quad(gigalitre):
  return gigalitre * gigalitres_to_cubic_metres * cubic_meter_to_quad

  #TODO: Fill out prod/imports/exports

## Supply 2019

In [69]:
energy_types = ['Hydro / Wave / Tidal', 'Natural Gas', 'Oil', 'Biomass / Geothermal', 'Coal & Coke', 'Uranium', 'Solar', 'Wind']

# Returns a dictionary with energy production by year in the following format:
# ```
# {
#   "2005": {
#     "Alberta": {
#       "Hydro / Wave / Tidal": value
#     },
#     ...
#   }
# }
# ```
def load_generation_data():
  energy_production = {}
  canada_energy_production = {}

  with open(f'{data_folder}/canada/electricity-generation-2019.csv') as csv_file:
    reader = csv.reader(csv_file)
    # Skip the header line
    next(reader)
  
    for row in reader:
      province = row[1]
      energy_type = row[2]
      year = row[4]
      quantity = float(row[5])

      # We don't want Canada-wide data (we can aggregate it ourselves)
      if province == "Canada":
        year_dictionary = get_nested_dictionary(canada_energy_production, year)
        year_dictionary[energy_type] = quantity
      else:
        year_dictionary = get_nested_dictionary(energy_production, year)
        province_dictionary = get_nested_dictionary(year_dictionary, province)
        province_dictionary[energy_type] = quantity
  return energy_production, canada_energy_production

def get_nested_dictionary(dictionary, key):
  value = dictionary.get(key)
  if value == None:
    dictionary[key] = {}
    value = dictionary[key]
  return value

def extract_energy_values(energy_production, year):
  flattened_values = []
  
  for row in list(energy_production[year].items()):
    energy = [row[0]]
    for energy_type in energy_types:
      energy.append(row[1][energy_type])
    flattened_values.append(energy)

  return np.array(flattened_values)

def style_axes(ax):
  # The following was copied from https://www.pythoncharts.com/matplotlib/grouped-bar-charts-matplotlib/
  ax.spines['top'].set_visible(False)
  ax.spines['right'].set_visible(False)
  ax.spines['left'].set_visible(False)
  ax.spines['bottom'].set_color('#DDDDDD')
  ax.tick_params(bottom=False, left=False)
  ax.set_axisbelow(True)
  ax.yaxis.grid(True, color='#EEEEEE')
  ax.xaxis.grid(False)

  # For each bar in the chart, add a text label so we can easily read the graph
  for bar in ax.patches:
    bar_value = bar.get_height()
    # Don't show any decimal values (they're big enough)
    text = f'{bar_value:.0f}'
    text_x = bar.get_x() + bar.get_width() / 2
    text_y = bar.get_y() + bar_value

    ax.text(text_x, text_y, text, ha='center', va='bottom', color=bar.get_facecolor(), size=12)

def plot_energy_by_province(title, energy_values):
  province_names = energy_values[:, 0]
  province_count = len(province_names)
  energy_type_count = len(energy_types)

  bar_width = 1.0
  full_width = energy_type_count * bar_width
  half_width = full_width / 2
  padding = bar_width * 2

  # This represents the location of the xticks (or the center of the 'bars') so we need to account for the full width of the bars, otherwise we overlap
  x = np.arange(province_count) * (full_width + padding)

  fig_width = bar_width * (province_count) * energy_type_count

  fig, ax = plt.subplots(figsize=(fig_width, 10))
  for i in range(energy_type_count):
    ax.bar(x - half_width + i * bar_width, energy_values[:, i + 1].astype(np.float), width=bar_width)

  style_axes(ax)

  plt.xticks(x - bar_width / 2, province_names)
  plt.ylabel('GWh Generated')
  plt.xlabel('Provinces')
  plt.title(title)
  plt.legend(energy_types)

  plt.show()

In [70]:
# Plot 2019 Data

energy_production, canada_energy_production = load_generation_data()
energy_values = extract_energy_values(energy_production, '2019')
plot_energy_by_province('2019 Energy Production - Canada', energy_values)

NameError: name 'data_folder' is not defined

## Demand 2019


## Emissions


## Solutions